In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()

1

In [50]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν = 2 * pi * 1.3
initial_state=kron([1.0 0 0 0;0 0 0 0;0 0 0 0;0 0 0 0],thermal_state(dim,1))

function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,4*dim,:)
end

#按照g,r,e,t的顺序
n=kron(eye(4),a₊(dim)*a₋(dim))
ee=kron([0 0 0 0;0 0 0 0;0 0 1 0;0 0 0 0],eye(dim))
rr=kron([0 0 0 0;0 1 0 0;0 0 0 0;0 0 0 0],eye(dim))
tt=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 0 0 1],eye(dim))

η=0.15
costheta = sqrt(2)/2
γg = 2 * pi * 6.89 
γr = 2 * pi * 13.79  

a=a₊(dim)+a₋(dim)
ge=kron([0 0 1 0;0 0 0 0;0 0 0 0;0 0 0 0],exp(im*η*costheta*a))
eg=kron([0 0 0 0;0 0 0 0;1 0 0 0;0 0 0 0],exp(-im*η*costheta*a))
u=ge+eg
rt=kron([0 0 0 0;0 0 0 1;0 0 0 0;0 0 0 0],exp(im*η*costheta*a))
t_r=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 1 0 0],exp(-im*η*costheta*a))
u2=rt+t_r
uu=u+u2
r_e=kron([0 0 0 0;0 0 1 0;0 0 0 0;0 0 0 0],exp(-im*η*costheta*a))
er=kron([0 0 0 0;0 0 0 0;0 1 0 0;0 0 0 0],exp(im*η*costheta*a))
u3=r_e+er

E=eye(4*dim)

function diss1()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;1 0 0 0;0 0 0 0],jump')
        adown=kron([0 0 1 0;0 0 0 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 1 0 0],jump')
        adown=kron([0 0 0 0;0 0 0 1;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss3()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 1.5 * (1 - theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 1 0 0;0 0 0 0],jump')
        adown=kron([0 0 0 0;0 0 1 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss4()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 1.5 * (1 - theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 0 0 0;1 0 0 0],jump')
        adown=kron([0 0 0 1;0 0 0 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
dissipation1=γg*diss1()
dissipation2=γr*diss2()
dissipation3=γr*diss3()
dissipation4=γg*diss4()
instate=dm2vec(initial_state);

In [51]:
function eit_Lindbladian_vec(ΩL1,ΩL2,Δg,Δr,Δt)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δg*ee+(Δr-Δg)*rr+(Δr-Δg-Δt)*tt+ΩL1/2*uu+ΩL2/2*u3))+kron(transpose(im*(ν*n-Δg*ee+(Δr-Δg)*rr+(Δr-Δg-Δt)*tt+ΩL1/2*uu+ΩL2/2*u3)),E)
    
    return unitary+dissipation1+dissipation2+dissipation3+dissipation4
end

eit_Lindbladian_vec (generic function with 1 method)

In [53]:
eit_vec=eit_Lindbladian_vec(2 * pi * 2.236, 2 * pi * 23.7 ,2 * pi * 108, 2 * pi * 108 , 2 * pi * 117.33)
list=[real(tr(n*vec2dm(instate)))]
delta_t=5
f=instate
evo=exp(delta_t*eit_vec)
@time for i in 1:100
    f=evo*f
    print(i," ")
    append!(list,real(tr(n*vec2dm(f)))) 
end
plot(0:5:500,list)
plot!(0:5:500,res2["n[2]"])

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100   0.302656 seconds (4.07 k allocations: 5.002 MiB)


<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
# Δg = 2 * pi * 108 
# Δr = 2 * pi * 108 
# Δt = 2 * pi * 117.33
# ΩL1 = 2 * pi * 2.236 
# ΩL2 = 2 * pi * 23.7 

ttot=200
function feval(x)
    f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],x[3]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end

function g!(G,seq)
    print("g!  ")
    a=real(gradient(feval,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end

In [36]:
function bosonic_thermal_state(d::Int, n::Real)
    q = n / (n+1)
    r = [q^j for j in 0:d-1]
    r ./= sum(r)
    v = zeros(d, d)
    for i in 1:d
        v[i, i] = r[i]
    end
    return v
end

function four_level_eit_cooling(d, Delta_g, Delta_r, Delta_t, nu, eta, costheta, omega_L1, omega_L2, gamma_g, gamma_r, nbar, t)
    ps = nlevel(["g", "r", "e", "t"])
    pb = boson(d=d)    
    xop = pb["a"] + pb["adag"]
    
    model = build_model([ps, pb], isunitary=false)
    add_unitary!(model, (1,), ("e<-e",), coeff=-Delta_g)
    add_unitary!(model, (1,), ("r<-r",), coeff=-Delta_g+Delta_r)
    add_unitary!(model, (1,), ("t<-t",), coeff=-Delta_g+Delta_r-Delta_t)
    add_unitary!(model, (2,), ("n",), coeff=nu)
    
    exp_xop = exp((im*eta*costheta)*xop)
    add_unitary!(model, (1, 2), ("g<-e", exp_xop), coeff=omega_L1/2)
    add_unitary!(model, (1, 2), ("e<-g", exp_xop'), coeff=omega_L1/2)
    add_unitary!(model, (1, 2), ("r<-t", exp_xop), coeff=omega_L1/2)
    add_unitary!(model, (1, 2), ("t<-r", exp_xop'), coeff=omega_L1/2)    

    add_unitary!(model, (1, 2), ("r<-e", exp_xop'), coeff=omega_L2/2)
    add_unitary!(model, (1, 2), ("e<-r", exp_xop), coeff=omega_L2/2)
    
    dx = 0.01
    for x in -1:dx:1
        c = dx * 0.75 * (1 + x^2)
        jump = sqrt(c / 2) * exp(-im*eta*x*xop)
        add_dissipation!(model, (1, 2), ("g<-e", jump), coeff=gamma_g/2)
    end   
    for x in -1:dx:1
        c = dx * 0.75 * (1 + x^2)
        jump = sqrt(c / 2) * exp(-im*eta*x*xop)
        add_dissipation!(model, (1, 2), ("r<-t", jump), coeff=gamma_r/2)
    end 

    for x in -1:dx:1
        c = dx * 1.5 * (1 - x^2)
        jump = sqrt(c / 2) * exp(-im*eta*x*xop)
        add_dissipation!(model, (1, 2), ("r<-e", jump), coeff=gamma_r/2)
    end
    for x in -1:dx:1
        c = dx * 1.5 * (1 - x^2)
        jump = sqrt(c / 2) * exp(-im*eta*x*xop)
        add_dissipation!(model, (1, 2), ("g<-t", jump), coeff=gamma_g/2)
    end

    
    # set observers
    add_observer!(model, (1,), ("e<-e",), name="e")
    add_observer!(model, (1,), ("g<-g",), name="g")
    add_observer!(model, (1,), ("r<-r",), name="r")
    add_observer!(model, (1,), ("t<-t",), name="t")
    add_observer!(model, (2,), ("n",), name="n")
    
    s1 = zeros(ComplexF64, 4,4)
    s1[1,1] = 1
    s2 = bosonic_thermal_state(d, nbar)

    state = tensor_product(s1, s2) 
    ham, observer = model.h, model.observer

    L = size(state, 1)
    state = reshape(state, L*L)

    ham = matrix(ham)
    observer = immrep(observer)
    result = measure(observer, reshape(state, L, L))
    obs = Observables(result)

    mt = 5
    n = round(Int, t/mt)
    for tj in 1:n
        print(tj," ")
        state = evolve(ham, mt, state, ishermitian=false)
        tmp = measure(observer, reshape(state, L, L))
        append!(obs, tmp)
    end
    return obs
end

nu = 2 * pi * 1.3
Delta_g = 2 * pi * 108 
Delta_r = 2 * pi * 108 
Delta_t = 2 * pi * 117.33
eta = 0.15

# Omega_L1 = 2 * pi * 1.
Omega_L2 = 2 * pi * 23.7 
Omega_L1 = 2 * pi * 2.236
costheta = sqrt(2)/2
gamma_g = 2 * pi * 6.89 
gamma_r = 2 * pi * 13.79  


d = 10
nave = 1.
 
@time res2=four_level_eit_cooling(d, Delta_g, Delta_r, Delta_t, nu, eta, costheta, Omega_L1, Omega_L2, gamma_g, gamma_r, nave, 500.)


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 105.584463 seconds (4.66 M allocations: 14.226 GiB, 3.01% gc time)


606-element Observables:
 ("norm", 1.0 + 0.0im)
 ("e[1]", 0.0)
 ("g[1]", 1.0)
 ("r[1]", 0.0)
 ("t[1]", 0.0)
 ("n[2]", 0.9902248289345063)
 ("norm", 1.0000000000000002 - 2.7264297842948408e-17im)
 ("e[1]", 0.0008001637485942567)
 ("g[1]", 0.9249848055315052)
 ("r[1]", 0.07420842470640014)
 ("t[1]", 6.606013500370579e-6)
 ("n[2]", 0.825658566084943)
 ("norm", 1.0000000000000004 + 7.947741118562879e-18im)
 ⋮
 ("norm", 0.9999999999999929 + 1.1115891952766111e-16im)
 ("e[1]", 9.056769854042868e-6)
 ("g[1]", 0.9909391110775644)
 ("r[1]", 0.009051016659623837)
 ("t[1]", 8.154929506936835e-7)
 ("n[2]", 0.00407061226421549)
 ("norm", 0.9999999999999926 + 1.1232526395621545e-16im)
 ("e[1]", 9.056767447348687e-6)
 ("g[1]", 0.9909391109614212)
 ("r[1]", 0.009051016771497816)
 ("t[1]", 8.154996265628431e-7)
 ("n[2]", 0.004070612263788733)